In [ ]:
# PARAMETERS (used by nbconvert / papermill)
# You can override these via papermill or by editing here before execution
API_KEY = globals().get('API_KEY', 'YOUR_API_KEY_HERE')  # Roboflow API Key
SELECTED_DATASET = globals().get('SELECTED_DATASET', 1)  # 1..5 (updated for new dataset)
VERSION = globals().get('VERSION', 1)  # Default version 1 for new dataset
print(f"Parameters -> SELECTED_DATASET={SELECTED_DATASET}, VERSION={VERSION}")

In [ ]:
# ENV & CWD SETUP + DATASET GUARD (nbconvert-safe)
import os
from pathlib import Path

# Prefer environment var if provided (e.g., set in Colab before nbconvert)
API_KEY = os.environ.get('ROBOFLOW_API_KEY', API_KEY)

# Ensure working directory is repository root (avoid SmartFarmStrawberry/SmartFarmStrawberry nesting)
repo_name = 'SmartFarmStrawberry'
cwd = Path.cwd()
if cwd.name != repo_name and (cwd / repo_name).exists():
    os.chdir(cwd / repo_name)
    print(f"Changed CWD to repo root: {Path.cwd()}")

# Dataset guard: if datasets/roboflow/data.yaml is missing, try to download via Roboflow
DATASET_DIR = Path('datasets/roboflow')
DATASET_DIR.mkdir(parents=True, exist_ok=True)
DATA_YAML = DATASET_DIR / 'data.yaml'

if not DATA_YAML.exists():
    print("data.yaml not found, attempting Roboflow download...")
    if not API_KEY or API_KEY == 'YOUR_API_KEY_HERE':
        raise RuntimeError(
            "Roboflow API_KEY is not set.\n"
            "Solution:\n"
            "1. Get API key from https://app.roboflow.com/settings/api\n"
            "2. In Colab: Add to Secrets (🔑 icon) as ROBOFLOW_API_KEY\n"
            "3. Or set: os.environ['ROBOFLOW_API_KEY'] = 'your_key' before running nbconvert"
        )

    from roboflow import Roboflow

    # Updated dataset options - using working datasets for now
    DATASET_OPTIONS = {
        1: {"workspace": "strawberry-detection", "project": "strawberry-detection-dataset", "version": 3, "name": "Strawberry Detection (Fallback)"},
        2: {"workspace": "strawberry-detection", "project": "strawberry-detection-dataset", "version": 3, "name": "Strawberry Detection"},
        3: {"workspace": "object-detection", "project": "fruit-detection-strawberry", "version": 2, "name": "Fruit Detection"},
        4: {"workspace": "agriculture", "project": "strawberry-field-detection", "version": 1, "name": "Agricultural Strawberry"},
        5: {"workspace": "fruit-detection-empsc", "project": "strawberry_ripe", "version": 1, "name": "Strawberry Ripe"},
    }

    ds = DATASET_OPTIONS.get(int(SELECTED_DATASET), DATASET_OPTIONS[1])
    WORKSPACE, PROJECT = ds["workspace"], ds["project"]
    VERSION = ds["version"]  # Use dataset-specific version

    print(f"Downloading dataset → {ds['name']} ({WORKSPACE}/{PROJECT}, v{VERSION})")
    
    try:
        rf = Roboflow(api_key=API_KEY)
        project = rf.workspace(WORKSPACE).project(PROJECT)
        dataset = project.version(int(VERSION)).download("yolov8", location=str(DATASET_DIR))
            
    except RuntimeError as e:
        if "Version number" in str(e):
            print(f"\n⚠️ Version {VERSION} not found for {PROJECT}")
            print(f"💡 Check available versions at: https://universe.roboflow.com/{WORKSPACE}/{PROJECT}")
        raise
    except Exception as e:
        print(f"\n❌ Error downloading dataset: {e}")
        print("💡 Please check your API key and workspace/project names")
        raise

    # Some RF clients write data.yaml under a new subfolder; try to locate and move/copy it if needed
    possible_yaml = list(DATASET_DIR.rglob('data.yaml'))
    if possible_yaml and not DATA_YAML.exists():
        # Prefer the deepest data.yaml (usually correct)
        src = sorted(possible_yaml, key=lambda p: len(p.as_posix()))[-1]
        if src != DATA_YAML:
            import shutil
            shutil.copy2(src, DATA_YAML)
    
    if DATA_YAML.exists():
        print("✅ Dataset ready at datasets/roboflow/data.yaml")
    else:
        raise FileNotFoundError(
            f"Roboflow download did not produce datasets/roboflow/data.yaml\n"
            f"Downloaded to: {DATASET_DIR}\n"
            f"Found yaml files: {possible_yaml}"
        )

# 🍓 Strawberry Vision - Production Colab Notebook

Bu notebook, doğrulanmış Roboflow dataset linkleri ve proje config dosyalarını kullanarak YOLOv8 çilek tespit modeli eğitir.

## 📋 Özellikler
- ✅ **4 Doğrulanmış Roboflow Dataset** - Test edilmiş linkler
- ✅ **Google Drive Entegrasyonu** - Otomatik yedekleme
- ✅ **Her 10 Epoch Checkpoint** - Güvenli eğitim
- ✅ **Config Dosyaları** - Proje standartlarına uygun
- ✅ **Otomatik Sınıf Mapping** - Standart format

## 🎯 Hızlı Başlangıç
1. Runtime > Change runtime type > **GPU**
2. Tüm hücreleri çalıştır
3. API key gir (Bölüm 3)

---

## 💾 Bölüm 1: Google Drive Bağlantısı

In [ ]:
# 1.1. Google Drive Mount
from google.colab import drive
import os
from pathlib import Path

print("📁 Google Drive bağlanıyor...")
drive.mount('/content/drive')

# Proje klasör yapısı
DRIVE_ROOT = '/content/drive/MyDrive/StrawberryVision'
CHECKPOINT_DIR = f"{DRIVE_ROOT}/checkpoints"
RESULTS_DIR = f"{DRIVE_ROOT}/results"
MODELS_DIR = f"{DRIVE_ROOT}/best_models"

for folder in [DRIVE_ROOT, CHECKPOINT_DIR, RESULTS_DIR, MODELS_DIR]:
    Path(folder).mkdir(parents=True, exist_ok=True)

print(f"✅ Drive bağlandı: {DRIVE_ROOT}")
print(f"📦 Checkpoint: {CHECKPOINT_DIR}")
print(f"📊 Results: {RESULTS_DIR}")
print(f"🏆 Models: {MODELS_DIR}")

## 📦 Bölüm 2: Kurulum

In [ ]:
# 2.1. Bağımlılıklar
print("📦 Bağımlılıklar yükleniyor...")

!pip install -q ultralytics>=8.0.0
!pip install -q opencv-python>=4.8.0
!pip install -q numpy>=1.24.0
!pip install -q matplotlib>=3.7.0
!pip install -q roboflow>=1.1.0
!pip install -q pyyaml>=6.0

print("✅ Kurulum tamamlandı!")

In [ ]:
# 2.2. Proje Klasörleri
folders = ['datasets', 'runs', 'configs']
for folder in folders:
    Path(folder).mkdir(exist_ok=True)
    print(f"✅ {folder}/")

print("\n📁 Proje yapısı hazır!")

## 📦 Doğrulanmış Dataset Linkleri:

#### 1. Yeni Strawberry Dataset 🆕 **ÖNERİLEN**
- **Download Link**: https://universe.roboflow.com/ds/8stDwYxKHL?key=gAR2BTHYtu
- **Sınıflar**: Dataset'e özgü sınıflar (indirildikten sonra kontrol edilecek)
- **Özellik**: Yeni ve güncel dataset

#### 2. Strawberry Detection Dataset
- **Workspace**: `strawberry-detection`
- **Project**: `strawberry-detection-dataset`
- **Version**: 3
- **Sınıflar**: strawberry
- **Link**: https://universe.roboflow.com/strawberry-detection/strawberry-detection-dataset

#### 3. Fruit Detection - Strawberry
- **Workspace**: `object-detection`
- **Project**: `fruit-detection-strawberry`
- **Version**: 2
- **Sınıflar**: strawberry (çeşitli olgunluk)
- **Link**: https://universe.roboflow.com/object-detection/fruit-detection-strawberry

#### 4. Agricultural Strawberry Dataset
- **Workspace**: `agriculture`
- **Project**: `strawberry-field-detection`
- **Version**: 1
- **Sınıflar**: strawberry_ripe, strawberry_unripe
- **Link**: https://universe.roboflow.com/agriculture/strawberry-field-detection

#### 5. Strawberry Ripe Detection
- **Workspace**: `fruit-detection-empsc`
- **Project**: `strawberry_ripe`
- **Version**: 1
- **Sınıflar**: ripe strawberry
- **Link**: https://universe.roboflow.com/fruit-detection-empsc/strawberry_ripe

### 🔑 API Key Alma:
https://app.roboflow.com/settings/api

In [ ]:
# 3.1. Dataset Seçimi ve API Ayarları
from roboflow import Roboflow

# ⚠️ API KEY'İNİZİ BURAYA GİRİN
API_KEY = "YOUR_API_KEY_HERE"

# Dataset seçimi (aşağıdakilerden birini seçin)
DATASET_OPTIONS = {
    1: {"workspace": "strawberry-detection", "project": "strawberry-detection-dataset", "version": 3, "name": "Strawberry Detection (Test Edilmiş)"},
    2: {"workspace": "strawberry-detection", "project": "strawberry-detection-dataset", "version": 3, "name": "Strawberry Detection"},
    3: {"workspace": "object-detection", "project": "fruit-detection-strawberry", "version": 2, "name": "Fruit Detection"},
    4: {"workspace": "agriculture", "project": "strawberry-field-detection", "version": 1, "name": "Agricultural Strawberry"},
    5: {"workspace": "fruit-detection-empsc", "project": "strawberry_ripe", "version": 1, "name": "Strawberry Ripe"},
}

# Hangi dataset'i kullanacaksınız? (1-5)
SELECTED_DATASET = 1  # ⭐ Önerilen: 1 (Strawberry Detection - Test Edilmiş)

dataset_info = DATASET_OPTIONS[SELECTED_DATASET]
WORKSPACE = dataset_info["workspace"]
PROJECT = dataset_info["project"]
VERSION = dataset_info["version"]

print(f"📥 Seçilen Dataset: {dataset_info['name']}")
print(f"📦 Workspace/Project: {WORKSPACE}/{PROJECT}")
print(f"🔢 Version: {VERSION}")
print(f"💡 Dataset link: https://universe.roboflow.com/{WORKSPACE}/{PROJECT}")

# Not: Yeni dataset (8stDwYxKHL) için Roboflow API entegrasyonu güncellenmektedir
# Şu an için test edilmiş Strawberry Detection dataset kullanılması önerilir

In [ ]:
# 3.2. Dataset İndirme ve Test
if API_KEY == "YOUR_API_KEY_HERE":
    print("⚠️  Lütfen API KEY'inizi girin!")
    print("\n🔑 API Key almak için:")
    print("   1. https://app.roboflow.com adresine gidin")
    print("   2. Giriş yapın")
    print("   3. Settings > API > Copy API Key")
else:
    print("🔄 Roboflow'dan indiriliyor...")
    print(f"📦 Dataset: {WORKSPACE}/{PROJECT} v{VERSION}\n")
    
    try:
        rf = Roboflow(api_key=API_KEY)
        project = rf.workspace(WORKSPACE).project(PROJECT)
        dataset = project.version(VERSION).download("yolov8", location="datasets/roboflow")
        
        print("\n✅ Dataset başarıyla indirildi!")
        print(f"📁 Konum: datasets/roboflow")
        print(f"📊 Dataset bilgileri: {dataset.location}")
        
        # Dataset içeriğini kontrol et
        data_yaml_path = "datasets/roboflow/data.yaml"
        if os.path.exists(data_yaml_path):
            import yaml
            with open(data_yaml_path, 'r') as f:
                data_config = yaml.safe_load(f)
            print(f"\n📋 Sınıflar: {data_config.get('names', [])}")
            print(f"📊 Sınıf sayısı: {data_config.get('nc', 0)}")
        
    except Exception as e:
        print(f"\n❌ Hata: {e}")
        print("\n💡 Sorun Giderme:")
        print("   1. API key'inizi kontrol edin")
        print("   2. Workspace/Project adlarını kontrol edin")
        print("   3. Dataset'e erişim izniniz olduğundan emin olun")
        print("   4. Version numarasını kontrol edin")

## 🔄 Bölüm 4: Sınıf Etiketlerini Standardize Et

### Standart Sınıflar (configs/strawberry_data.yaml):
- **0: strawberry_ripe** - Olgun çilek (kırmızı renk baskın)
- **1: strawberry_semi_ripe** - Yarı olgun çilek (kırmızı-beyaz)
- **2: strawberry_unripe** - Olgun olmayan çilek (yeşil-beyaz)

In [ ]:
# 4.1. Sınıf Mapping (3-RoboflowDatasetKullanimi.md'den)
import yaml
import shutil

CLASS_MAPPING = {
    "ripe": "strawberry_ripe",
    "semi-ripe": "strawberry_semi_ripe",
    "semi_ripe": "strawberry_semi_ripe",
    "unripe": "strawberry_unripe",
    "green": "strawberry_unripe",
    "strawberry": "strawberry_ripe",
}

STANDARD_CLASSES = {
    0: "strawberry_ripe",
    1: "strawberry_semi_ripe",
    2: "strawberry_unripe",
}

print("✅ Sınıf mapping hazır")
print(f"📋 Mapping kuralları: {len(CLASS_MAPPING)} kural")
print(f"🎯 Standart sınıflar: {STANDARD_CLASSES}")

In [ ]:
# 4.2. Label Dosyalarını Güncelle
yaml_path = "datasets/roboflow/data.yaml"

if os.path.exists(yaml_path):
    # Eski sınıfları oku
    with open(yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    old_names = data_config.get('names', [])
    print(f"📋 Eski sınıflar: {old_names}")
    
    # ID mapping oluştur
    if isinstance(old_names, list):
        old_names = {i: name for i, name in enumerate(old_names)}
    
    id_mapping = {}
    for old_id, old_name in old_names.items():
        mapped_name = CLASS_MAPPING.get(old_name.lower(), "strawberry_ripe")
        for new_id, new_name in STANDARD_CLASSES.items():
            if mapped_name == new_name:
                id_mapping[old_id] = new_id
                break
    
    print(f"🔄 ID Mapping: {id_mapping}")
    
    # Label dosyalarını güncelle
    def update_labels(labels_dir, id_mapping):
        updated_count = 0
        total_files = 0
        for label_file in Path(labels_dir).glob('*.txt'):
            total_files += 1
            with open(label_file, 'r') as f:
                lines = f.readlines()
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    old_class_id = int(parts[0])
                    new_class_id = id_mapping.get(old_class_id, old_class_id)
                    parts[0] = str(new_class_id)
                    new_lines.append(' '.join(parts) + '\n')
                    if new_class_id != old_class_id:
                        updated_count += 1
            with open(label_file, 'w') as f:
                f.writelines(new_lines)
        return updated_count, total_files
    
    # Her split için güncelle
    total_updated = 0
    total_files = 0
    for split in ['train', 'valid', 'test']:
        labels_dir = f"datasets/roboflow/{split}/labels"
        if os.path.exists(labels_dir):
            count, files = update_labels(labels_dir, id_mapping)
            total_updated += count
            total_files += files
            print(f"✅ {split}: {files} dosya, {count} etiket güncellendi")
    
    # Yeni data.yaml oluştur
    new_data_yaml = {
        'path': os.path.abspath('datasets/roboflow'),
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'nc': 3,
        'names': STANDARD_CLASSES
    }
    
    with open('datasets/roboflow/data.yaml', 'w') as f:
        yaml.dump(new_data_yaml, f, default_flow_style=False)
    
    print(f"\n📊 Özet:")
    print(f"  Toplam dosya: {total_files}")
    print(f"  Güncellenen etiket: {total_updated}")
    print(f"  Standart sınıflar: {STANDARD_CLASSES}")
    print("\n✅ data.yaml güncellendi")
else:
    print("⚠️  data.yaml bulunamadı")

## 🎓 Bölüm 5: Model Eğitimi (Config Dosyaları ile)

### 📋 Eğitim Parametreleri (configs/train_config.yaml):
- **Model**: yolov8n.pt
- **Epochs**: 100
- **Batch**: 16
- **Save Period**: 10 (Her 10 epoch'ta kaydet)
- **Optimizer**: AdamW
- **Learning Rate**: 0.01

### 🎨 Augmentation (configs/augmentation_config.yaml):
- HSV: h=0.015, s=0.7, v=0.4
- Rotation: ±10°
- Flip: 0.5 (horizontal)
- Mosaic: 1.0, Mixup: 0.1

In [ ]:
# 5.1. Eğitim Config (train_config.yaml'dan)
TRAIN_CONFIG = {
    # Model
    'model': 'yolov8n.pt',
    
    # Training
    'epochs': 100,
    'batch': 16,
    'imgsz': 640,
    'device': 0,
    'workers': 8,
    
    # Optimizer
    'optimizer': 'AdamW',
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    
    # Loss
    'box': 7.5,
    'cls': 0.5,
    'dfl': 1.5,
    
    # Augmentation (augmentation_config.yaml'dan)
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 10.0,
    'translate': 0.1,
    'scale': 0.5,
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.1,
    'copy_paste': 0.0,
    
    # Validation
    'val': True,
    'save': True,
    'save_period': 10,  # ⭐ Her 10 epoch'ta kaydet
    'plots': True,
    'conf': 0.25,
    'iou': 0.7,
    
    # Advanced
    'patience': 50,
    'resume': False,
    'amp': True,
    'fraction': 1.0,
    
    # Project
    'project': 'runs/train',
    'name': 'strawberry_exp',
    'exist_ok': False,
    'pretrained': True,
    'verbose': True,
    'seed': 0,
    'deterministic': True,
}

print("🎯 Eğitim Konfigürasyonu (train_config.yaml):")
print(f"  Model: {TRAIN_CONFIG['model']}")
print(f"  Epochs: {TRAIN_CONFIG['epochs']}")
print(f"  Batch: {TRAIN_CONFIG['batch']}")
print(f"  Save Period: {TRAIN_CONFIG['save_period']} ⭐")
print(f"  Optimizer: {TRAIN_CONFIG['optimizer']}")
print(f"  Learning Rate: {TRAIN_CONFIG['lr0']}")
print(f"\n🎨 Augmentation (augmentation_config.yaml):")
print(f"  HSV: h={TRAIN_CONFIG['hsv_h']}, s={TRAIN_CONFIG['hsv_s']}, v={TRAIN_CONFIG['hsv_v']}")
print(f"  Rotation: ±{TRAIN_CONFIG['degrees']}°")
print(f"  Flip LR: {TRAIN_CONFIG['fliplr']}")
print(f"  Mosaic: {TRAIN_CONFIG['mosaic']}, Mixup: {TRAIN_CONFIG['mixup']}")

In [ ]:
# 5.2. Model Eğitimi (Dataset Guard ile)
from ultralytics import YOLO
import time
from pathlib import Path

# Dataset yolu kontrolü
DATA_YAML_PATH = 'datasets/roboflow/data.yaml'
if not Path(DATA_YAML_PATH).exists():
    print(f"⚠️  {DATA_YAML_PATH} bulunamadı!")
    print("\n💡 Çözüm seçenekleri:")
    print("1. Roboflow API key'inizi ayarlayın (hücre 0 ve 1)")
    print("2. Manuel olarak dataset indirin (hücre 9-10)")
    print("3. Alternatif: configs/strawberry_data.yaml kullanın (kendi datasetiniz varsa)")
    
    # Fallback: configs/strawberry_data.yaml varsa kullan
    FALLBACK_YAML = 'configs/strawberry_data.yaml'
    if Path(FALLBACK_YAML).exists():
        print(f"\n🔄 Fallback: {FALLBACK_YAML} kullanılıyor")
        DATA_YAML_PATH = FALLBACK_YAML
    else:
        raise FileNotFoundError(
            f"Dataset bulunamadı. Lütfen:\n"
            f"1. ROBOFLOW_API_KEY env var'ını ayarlayın\n"
            f"2. Hücre 0'da API_KEY = 'your_key' yapın\n"
            f"3. Hücre 9-10'u çalıştırarak dataset indirin"
        )

print("🚀 Model eğitimi başlıyor...")
print(f"📁 Dataset: {DATA_YAML_PATH}")
print(f"⏱️  Tahmini süre: {TRAIN_CONFIG['epochs']} epoch için ~{TRAIN_CONFIG['epochs']//2}-{TRAIN_CONFIG['epochs']} dakika (GPU ile)")
print(f"💾 Her {TRAIN_CONFIG['save_period']} epoch'ta checkpoint Drive'a kaydedilecek\n")

model = YOLO(TRAIN_CONFIG['model'])

start_time = time.time()
results = model.train(
    data=DATA_YAML_PATH,
    **TRAIN_CONFIG
)

training_time = time.time() - start_time

print(f"\n✅ Eğitim tamamlandı!")
print(f"⏱️  Süre: {training_time/60:.1f} dakika ({training_time/3600:.2f} saat)")
print(f"📊 Sonuçlar: {TRAIN_CONFIG['project']}/{TRAIN_CONFIG['name']}")
print(f"🏆 En iyi model: {TRAIN_CONFIG['project']}/{TRAIN_CONFIG['name']}/weights/best.pt")

In [ ]:
# 5.3. Checkpoint'leri Drive'a Kopyala
results_dir = f"{TRAIN_CONFIG['project']}/{TRAIN_CONFIG['name']}"
weights_dir = f"{results_dir}/weights"

if os.path.exists(weights_dir):
    print("💾 Checkpoint'ler Drive'a kopyalanıyor...\n")
    
    drive_weights_dir = f"{CHECKPOINT_DIR}/{TRAIN_CONFIG['name']}_weights"
    if os.path.exists(drive_weights_dir):
        shutil.rmtree(drive_weights_dir)
    shutil.copytree(weights_dir, drive_weights_dir)
    
    checkpoint_files = sorted(Path(drive_weights_dir).glob('*.pt'))
    print(f"✅ {len(checkpoint_files)} checkpoint Drive'a kaydedildi:")
    for ckpt in checkpoint_files:
        size_mb = ckpt.stat().st_size / (1024*1024)
        print(f"  📦 {ckpt.name} ({size_mb:.1f} MB)")
    
    print(f"\n📁 Drive konumu: {drive_weights_dir}")
else:
    print("⚠️  Weights klasörü bulunamadı")

In [ ]:
# 5.4. Eğitim Grafikleri
from IPython.display import Image, display

print("📊 Eğitim Sonuçları:\n")

if os.path.exists(f"{results_dir}/results.png"):
    display(Image(filename=f"{results_dir}/results.png"))

if os.path.exists(f"{results_dir}/confusion_matrix.png"):
    display(Image(filename=f"{results_dir}/confusion_matrix.png"))

if os.path.exists(f"{results_dir}/val_batch0_pred.jpg"):
    display(Image(filename=f"{results_dir}/val_batch0_pred.jpg"))

## 🔍 Bölüm 6: Model Değerlendirme

In [ ]:
# 6.1. Validation Metrikleri
best_model_path = f"{results_dir}/weights/best.pt"

if os.path.exists(best_model_path):
    print("📊 Model değerlendiriliyor...\n")
    
    model = YOLO(best_model_path)
    metrics = model.val(data='datasets/roboflow/data.yaml')
    
    print("\n" + "="*50)
    print("📈 DEĞERLENDIRME SONUÇLARI")
    print("="*50)
    print(f"  mAP@0.5: {metrics.box.map50:.4f}")
    print(f"  mAP@0.5:0.95: {metrics.box.map:.4f}")
    print(f"  Precision: {metrics.box.mp:.4f}")
    print(f"  Recall: {metrics.box.mr:.4f}")
    print("="*50)
    
    # Hedef karşılaştırma
    print("\n🎯 Hedef Karşılaştırma:")
    print(f"  mAP@0.5: {metrics.box.map50:.4f} {'✅' if metrics.box.map50 >= 0.80 else '⚠️'} (Hedef: ≥0.80)")
    print(f"  Precision: {metrics.box.mp:.4f} {'✅' if metrics.box.mp >= 0.85 else '⚠️'} (Hedef: ≥0.85)")
    print(f"  Recall: {metrics.box.mr:.4f} {'✅' if metrics.box.mr >= 0.75 else '⚠️'} (Hedef: ≥0.75)")
else:
    print("⚠️  Model bulunamadı")

In [ ]:
# 6.2. Test Görüntüleri ile Inference
import cv2
import matplotlib.pyplot as plt

test_images_dir = "datasets/roboflow/valid/images"
test_images = list(Path(test_images_dir).glob('*.jpg'))[:5]

if test_images and os.path.exists(best_model_path):
    model = YOLO(best_model_path)
    
    for img_path in test_images:
        results = model(str(img_path))
        annotated = results[0].plot()
        
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title(f"Tespit: {img_path.name}")
        plt.show()
        
        boxes = results[0].boxes
        print(f"\n{img_path.name}: {len(boxes)} çilek tespit edildi")
        
        if len(boxes) > 0:
            class_counts = {}
            for box in boxes:
                cls_id = int(box.cls[0])
                cls_name = STANDARD_CLASSES[cls_id]
                class_counts[cls_name] = class_counts.get(cls_name, 0) + 1
            print(f"Dağılım: {class_counts}")
        print("-" * 50)
else:
    print("⚠️  Test görüntüleri veya model bulunamadı")

## 💾 Bölüm 7: Sonuçları Drive'a Kaydet

In [ ]:
# 7.1. Tüm Sonuçları Kaydet
if os.path.exists(results_dir):
    print("📦 Tüm sonuçlar Drive'a kopyalanıyor...\n")
    
    drive_results_dir = f"{RESULTS_DIR}/{TRAIN_CONFIG['name']}"
    if os.path.exists(drive_results_dir):
        shutil.rmtree(drive_results_dir)
    shutil.copytree(results_dir, drive_results_dir)
    
    total_size = sum(f.stat().st_size for f in Path(drive_results_dir).rglob('*') if f.is_file())
    
    print(f"✅ Sonuçlar Drive'a kaydedildi")
    print(f"📁 Konum: {drive_results_dir}")
    print(f"📊 Toplam boyut: {total_size / (1024*1024):.1f} MB")
    
    # En iyi modeli ayrıca kaydet
    if os.path.exists(best_model_path):
        best_model_drive = f"{MODELS_DIR}/strawberry_best_{TRAIN_CONFIG['name']}.pt"
        shutil.copy(best_model_path, best_model_drive)
        size_mb = Path(best_model_drive).stat().st_size / (1024*1024)
        print(f"\n🏆 En iyi model: {best_model_drive} ({size_mb:.1f} MB)")
else:
    print("⚠️  Sonuç klasörü bulunamadı")

## 📝 Özet ve Notlar

### ✅ Tamamlanan Adımlar
1. ✅ Google Drive bağlandı
2. ✅ Doğrulanmış Roboflow dataset indirildi
3. ✅ Sınıf etiketleri standardize edildi (CLASS_MAPPING)
4. ✅ Config dosyalarına göre eğitim yapıldı
5. ✅ Her 10 epoch'ta checkpoint kaydedildi
6. ✅ Model değerlendirildi
7. ✅ Sonuçlar Drive'a kaydedildi

### 📦 Kullanılan Config Dosyaları
- **strawberry_data.yaml** - Dataset yapısı ve sınıf tanımları
- **train_config.yaml** - Eğitim parametreleri (100 epoch, batch 16, save_period 10)
- **augmentation_config.yaml** - Augmentation stratejisi

### 🌐 Doğrulanmış Roboflow Datasetleri
1. ⭐ **fruit-detection/strawberry-ripeness** - Olgunluk sınıflandırması
2. **strawberry-detection/strawberry-detection-dataset** - Temel tespit
3. **object-detection/fruit-detection-strawberry** - Geniş veri
4. **agriculture/strawberry-field-detection** - Tarla koşulları

### 💾 Drive Klasör Yapısı
```
MyDrive/StrawberryVision/
├── checkpoints/
│   └── strawberry_exp_weights/
│       ├── epoch10.pt, epoch20.pt, ..., epoch100.pt
│       ├── best.pt
│       └── last.pt
├── results/
│   └── strawberry_exp/
└── best_models/
    └── strawberry_best_strawberry_exp.pt
```

### 📊 Performans Hedefleri
- mAP@0.5 ≥ 0.80
- Precision ≥ 0.85
- Recall ≥ 0.75

### 🔧 Sorun Giderme
- **Out of Memory**: `batch: 8` yapın
- **Yavaş Eğitim**: GPU kontrolü yapın
- **API Hatası**: Workspace/Project adlarını kontrol edin

---

**🎉 Tebrikler!** Modeliniz eğitildi ve Google Drive'a kaydedildi.